In [1]:
import os
import json
import datetime
import numpy as np
import pandas as pd
import lightgbm as lgb
import seaborn as sns
from ast import literal_eval
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier, MLPRegressor


import category_encoders as ce
from sklearn import preprocessing

%matplotlib inline
pd.options.display.max_columns = 999

In [5]:
def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    df['weekend'] = df['date'].apply(lambda x: x.weekday())
    
    return df

In [6]:
train_df = pd.read_csv('final\\train.csv', dtype={'fullVisitorId': 'str'})

## Feature Engineering

In [7]:
# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0, inplace=True)

# label encode the categorical variables and convert the numerical variables to float 
# scikit.rf needs numerical data. One hot encoding is not good on rf.
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

#these columns should be numbers
num_cols = ["fullVisitorId", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits', 'totals.transactionRevenue']    


#ordinal encoding
encoder = ce.OrdinalEncoder(cols=cat_cols)

train_df = encoder.fit_transform(train_df, train_df["totals.transactionRevenue"])

for col in num_cols:
    train_df[col] = train_df[col].astype(float)

In [8]:
cols_to_remove = ['totals.sessionQualityDim', 'totals.timeOnSite', 'totals.totalTransactionRevenue', 'totals.transactions', 'fullVisitorId', 'visitId']
train_df.drop(cols_to_remove, axis=1, inplace=True)

In [9]:
train_df.describe()

,channelGrouping,date,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.transactionRevenue,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,visitNumber,visitStartTime
count,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,871578.0,1.708337e+06,1307430.0,1.708098e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06,1.708337e+06
mean,3.181249e+00,2.017016e+07,3.268186e+00,1.352495e+00,2.984312e+00,3.149186e+01,2.117982e+00,1.896126e+01,3.135793e+00,1.614638e+03,1.933759e+01,5.772060e+00,1.0,4.429598e+00,1.0,3.696202e+00,1.355906e+06,2.107075e+00,1.093499e+00,1.224693e+03,1.044063e+00,9.004312e-02,1.069177e+00,1.342838e+00,1.312888e+00,2.348832e+01,2.584999e+00,9.054825e+01,7.627207e+00,2.335170e+00,1.498352e+09
std,2.338352e+00,6.485620e+03,5.089240e+00,5.521176e-01,1.748305e+00,8.578939e+01,8.427465e-01,2.214966e+01,7.055477e+00,5.327210e+03,4.787271e+01,3.249533e+00,0.0,8.991748e+00,0.0,6.473237e+00,4.522809e+07,6.137190e+00,4.827646e-01,6.707349e+03,2.052346e-01,4.259445e-01,3.386900e-01,1.909708e+00,4.636693e-01,1.809326e+02,1.306445e+00,2.950553e+02,1.330149e+01,9.354034e+00,1.624937e+07
min,1.000000e+00,2.016080e+07,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.470035e+09
25%,1.000000e+00,2.016122e+07,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,6.000000e+00,1.000000e+00,7.000000e+00,1.000000e+00,4.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.482738e+09
50%,3.000000e+00,2.017071e+07,2.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,6.000000e+00,1.000000e+00,4.800000e+01,1.000000e+00,5.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.499832e+09
75%,5.000000e+00,2.017120e+07,3.000000e+00,2.000000e+00,4.000000e+00,2.500000e+01,3.000000e+00,3.000000e+01,2.000000e+00,3.170000e+02,1.500000e+01,6.000000e+00,1.0,4.000000e+00,1.0,4.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,8.000000e+00,1.000000e+00,1.512513e+09
max,8.000000e+00,2.018043e+07,1.290000e+02,3.000000e+00,2.400000e+01,9.560000e+02,6.000000e+00,2.280000e+02,1.230000e+02,4.198200e+04,4.830000e+02,2.300000e+01,1.0,5.000000e+02,1.0,5.000000e+02,2.312950e+10,7.700000e+01,4.000000e+00,5.900900e+04,2.000000e+00,1.200000e+01,4.000000e+00,3.300000e+01,2.000000e+00,4.547000e+03,7.000000e+00,3.197000e+03,3.450000e+02,4.570000e+02,1.525158e+09


In [ ]:
#scaler = preprocessing.StandardScaler()
#scaled_df = scaler.fit_transform(train_df)

In [10]:
train_df = add_time_features(train_df)

### Train validation split

In [11]:
# Get labels
y_train = train_df['totals.transactionRevenue'].values
train_df.drop(['totals.transactionRevenue'], axis=1, inplace=True)
# Log transform the labels
y_train = np.log1p(y_train)

In [12]:
# drop date and id columns
train_df.drop(['date'], axis=1, inplace=True)

In [13]:
train_df.drop(['visitStartTime','totals.pageviews', 'totals.newVisits', 'totals.bounces'], axis=1, inplace=True)

In [14]:
features = train_df.columns.values
print('TRAIN SET')
print('Rows: %s' % train_df.shape[0])
print('Columns: %s' % train_df.shape[1])
print('Features: %s' % train_df.columns.values)

TRAIN SET
Rows: 1708337
Columns: 31
Features: ['channelGrouping' 'device.browser' 'device.deviceCategory'
 'device.isMobile' 'device.operatingSystem' 'geoNetwork.city'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.metro'
 'geoNetwork.networkDomain' 'geoNetwork.region' 'geoNetwork.subContinent'
 'totals.hits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsClickInfo.gclId'
 'trafficSource.adwordsClickInfo.isVideoAd'
 'trafficSource.adwordsClickInfo.page'
 'trafficSource.adwordsClickInfo.slot' 'trafficSource.campaign'
 'trafficSource.isTrueDirect' 'trafficSource.keyword'
 'trafficSource.medium' 'trafficSource.referralPath'
 'trafficSource.source' 'visitNumber' 'year' 'month' 'day' 'weekday'
 'weekend']


### Start modelling

In [15]:
model = MLPRegressor(activation='tanh',solver='adam',
    hidden_layer_sizes=(20,),
    max_iter=10000,
    shuffle=False,
    random_state=42)
print('model created')

model created


In [16]:
%%time

model.fit(train_df, y_train)

Wall time: 5min 18s


MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(20,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=42, shuffle=False, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [17]:
test_df = pd.read_csv('final\\test.csv', dtype={'fullVisitorId': 'str'})

In [18]:
#apply encoing
test_df = encoder.transform(test_df)

for col in num_cols:
    test_df[col] = test_df[col].astype(float)

In [19]:
# extract fullVisitorId before removing it


result_df = pd.DataFrame({"fullVisitorId":test_df["fullVisitorId"].values})

In [20]:
test_df = add_time_features(test_df)

In [21]:
test_df.drop(['visitStartTime','totals.pageviews', 'totals.newVisits', 'totals.bounces'], axis=1, inplace=True)

In [22]:
y_true = test_df['totals.transactionRevenue']

additional_cols_remove = ['totals.transactionRevenue', 'date', 'fullVisitorId']
# drop
test_df.drop(cols_to_remove + additional_cols_remove, axis=1, inplace=True)

In [23]:
print('TEST SET')
print('Rows: %s' % test_df.shape[0])
print('Columns: %s' % test_df.shape[1])
print('Features: %s' % test_df.columns.values)

TEST SET
Rows: 401589
Columns: 31
Features: ['channelGrouping' 'visitNumber' 'device.browser' 'device.deviceCategory'
 'device.isMobile' 'device.operatingSystem' 'geoNetwork.city'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.metro'
 'geoNetwork.networkDomain' 'geoNetwork.region' 'geoNetwork.subContinent'
 'totals.hits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsClickInfo.gclId'
 'trafficSource.adwordsClickInfo.isVideoAd'
 'trafficSource.adwordsClickInfo.page'
 'trafficSource.adwordsClickInfo.slot' 'trafficSource.campaign'
 'trafficSource.isTrueDirect' 'trafficSource.keyword'
 'trafficSource.medium' 'trafficSource.referralPath'
 'trafficSource.source' 'year' 'month' 'day' 'weekday' 'weekend']


In [24]:
%%time
predictions = model.predict(test_df)

Wall time: 1.54 s


In [25]:
rms = np.sqrt(mean_squared_error(y_true, predictions))

In [26]:
rms

59039153.911819585

In [27]:
predictions[predictions<0] = 0
result_df["transactionRevenue"] = y_true.values
result_df["PredictedRevenue"] = np.expm1(predictions)

result_df = result_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(mean_squared_error(np.log1p(result_df["transactionRevenue"].values), np.log1p(result_df["PredictedRevenue"].values))))

2.8051690231232733
